In [ ]:
import pandas as pd
import re 
import json

In [ ]:
df = pd.read_csv("career_data.csv")
df.head()

In [ ]:
df[['Degree',"Discipline"]] = df['Degree'].str.extract(r'(\w+)\s+(.*)')

In [ ]:
df_reorder = df.iloc[:,[0,9,1,2,3,4,5,6,7,8]]
df_reorder.head()

In [ ]:

def parse_exp(exp):
    if pd.isna(exp) or exp.lower() == 'none':
        return 0.0

    if 'internship' in exp: 
        return 0.5
        
    match = re.search(r'(\d+(\.\d+)?)',exp)
    
    if match: 
        return float(match.group(1))
    
    return 0.0

df_reorder['Experience'] = df_reorder['Experience'].apply(parse_exp)

In [ ]:
def fin_sup_convert(text):
    if text == "Higher":
        return 1.0
    if text == "Low":
        return 0.0
    return 0.5

df_reorder['Financial Support']= df_reorder['Financial Support'].apply(fin_sup_convert)

In [ ]:
df_reorder.to_csv("cleaned_data.csv",index=False)

In [ ]:
data = pd.read_csv("cleaned_data.csv")

with open("fine_tune_data.jsonl",'w',encoding="utf-8") as f :

  for _,row in data.iterrows():
    input_text = (
            f"Degree: {row['Degree']}, "
            f"Discipline: {row['Discipline']}, "
            f"Skills: {row['Skills']}, "
            f"Experience: {row['Experience']}, "
            f"CGPA: {row['CGPA']}, "
            f"Competitive Exam: {row['Competitive Exam']}, "
            f"Interest: {row['Interest']}, "
            f"Financial Support: {row['Financial Support']}"
        )
    
    output_text = row['Advice']

    json.dump({'input': input_text, "output": output_text},f,ensure_ascii=False)
    f.write('\n')
  print("JSON created successfully")
